In [1]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import wordnet as wn
import gensim 
from gensim.models import Word2Vec 
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [3]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

In [56]:
from nltk.corpus import stopwords
breakers = {'with', 'without', 'comma', 'and', 'or', 'by', 'in', 'due', 'to', 'of', 'causing'}
stop_words= (set(stopwords.words("english")) | {'nan', 'unspecified', 'stated', 'other', 'state'})

In [5]:
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()

In [6]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

In [70]:
from sklearn.preprocessing import OneHotEncoder
import itertools

In [7]:
df_train = np.asarray(pd.read_csv("data/diabetes/diab_train.csv", skiprows=1, header=None))
df_val = np.asarray(pd.read_csv("data/diabetes/diab_validation.csv", skiprows=1, header=None))
df_test = np.asarray(pd.read_csv("data/diabetes/diab_test.csv", skiprows=1, header=None))

In [8]:
X = np.hstack([df_train[:,:-4], df_train[:,-3:]])
X_val = np.hstack([df_val[:,:-4], df_val[:,-3:]])
X_test = np.hstack([df_test[:,:-4], df_test[:,-3:]])
Y = df_train[:,-4]
Y_val = df_val[:,-4]
Y_test = df_test[:,-4]

In [9]:
X_all = np.vstack([X, X_val, X_test])

In [71]:
def sanitize(doc):
    return [lem.lemmatize(w) for w in tokenizer.tokenize(str(doc).replace(",", " comma").lower())\
                     if w not in (stop_words - breakers)]

In [72]:
def sanitize2(doc):
    return [lem.lemmatize(w) for w in tokenizer.tokenize(str(doc).replace(",", " comma").lower())\
                     if w not in stop_words]

In [92]:
def onehot(tokens_docs):
    # convert list of of token-lists to one flat list of tokens
    # and then create a dictionary that maps word to id of word,
    all_tokens = itertools.chain.from_iterable(tokens_docs)
    word_to_id = {token: idx for idx, token in enumerate(set(all_tokens))}

    # convert token lists to token-id lists
    token_ids = [[word_to_id[token] for token in tokens_doc] for tokens_doc in tokens_docs]

    maxl = 0
    maxi = 0
    for t in token_ids:
        maxl = max(maxl, len(t))
        for w in t:
            maxi = max(maxi, w)

    enc = np.zeros((len(token_ids), maxl, maxi+1))
    for i,t in enumerate(token_ids):
        for j, w in enumerate(t):
            enc[i][j][w] += 1

    return enc

Race

In [59]:
print(set(X_all[:,1]))

{'Other', 'Asian', 'Caucasian', 'Hispanic', 'AfricanAmerican', '?'}


In [60]:
race = {'Other': 1, 'Asian':2, 'Caucasian':3, 'Hispanic':4, 'AfricanAmerican':5, '?':0}

Gender

In [61]:
print(set(X_all[:,2]))

{'Male', 'Female'}


In [62]:
gender = {'Male': 0, 'Female': 1}

Age

In [63]:
print(set(X_all[:,3]))

{'[0-10)', '[80-90)', '[60-70)', '[50-60)', '[30-40)', '[20-30)', '[90-100)', '[40-50)', '[70-80)', '[10-20)'}


In [64]:
gender = {'[0-10)': 0, '[80-90)': 8, '[60-70)': 6, '[50-60)': 5,\
          '[30-40)': 3, '[20-30)':2, '[90-100)':9, '[40-50)':4, '[70-80)': 7, '[10-20)': 10}

Weight

In [65]:
print(set(X_all[:,4]))

{'[25-50)', '[150-175)', '[75-100)', '[0-25)', '[125-150)', '[50-75)', '[100-125)', '?'}


In [66]:
weight = {'[25-50)': 25, '[150-175)': 150, '[75-100)': 75,\
          '[0-25)': 0, '[125-150)': 125, '[50-75)': 50, '[100-125)':100, '?': 73}

Admission Type ID

In [67]:
print(set(X_all[:,5]))

{nan, 'Newborn', 'Emergency', 'Not Mapped', 'Not Available', 'Urgent', 'Elective'}


In [68]:
admission_type = {'nan': 0, 'Newborn': 2, 'Emergency':4,\
                  'Not Mapped': 0, 'Not Available': 0, 'Urgent': 3, 'Elective': 1}

Discharge Disposition ID

In [99]:
print(len(set(X_all[:,6])), 'Items')
print(sorted(list(set(X_all[:,6].astype(str)))))

22 Items
['Admitted as an inpatient to this hospital', 'Discharged to home', 'Discharged/transferred to ICF', 'Discharged/transferred to SNF', 'Discharged/transferred to a federal health care facility.', 'Discharged/transferred to a long term care hospital.', 'Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare', 'Discharged/transferred to another  type of inpatient care institution', 'Discharged/transferred to another rehab fac including rehab units of a hospital.', 'Discharged/transferred to another short term hospital', 'Discharged/transferred to home under care of Home IV provider', 'Discharged/transferred to home with home health service', 'Discharged/transferred within this institution to Medicare approved swing bed', 'Discharged/transferred/referred another institution for outpatient services', 'Discharged/transferred/referred to a psychiatric hospital of a psychiatric distinct part unit of a hospital', 'Discharged/transferred/r

In [104]:
discharge_disposition = sorted([sanitize2(s) for s in set(X_all[:,6].astype(str))])
discharge_disposition_onehot = onehot(discharge_disposition)

Admission Source ID

In [110]:
print(sorted(list(set(X_all[:,7].astype(str)))))

['Clinic Referral', 'Court/Law Enforcement', 'Emergency Room', 'HMO Referral', 'Not Available', 'Not Mapped', 'Physician Referral', 'Transfer from a Skilled Nursing Facility (SNF)', 'Transfer from a hospital', 'Transfer from another health care facility', 'nan']


In [109]:
admission_src_id = {k: i for i,k in enumerate(sorted(list(set(X_all[:,7].astype(str)))))}

Time in Hospital

In [18]:
print(set(X_all[:,8]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}


Payer Code

In [19]:
print(set(X_all[:,9]))

{'DM', 'OT', 'CH', 'CP', 'MD', 'SI', 'WC', 'OG', 'BC', 'UN', 'HM', 'PO', 'MC', 'SP', '?', 'CM'}


In [106]:
payer_code = {k:i for i,k in enumerate(set(X_all[:,9]))}

Medical Speciality

In [101]:
print(sorted([s for s in set(X_all[:,10])]))

['?', 'Anesthesiology-Pediatric', 'Cardiology', 'Cardiology-Pediatric', 'Emergency/Trauma', 'Endocrinology', 'Family/GeneralPractice', 'Gastroenterology', 'Gynecology', 'Hematology', 'Hematology/Oncology', 'Hospitalist', 'InfectiousDiseases', 'InternalMedicine', 'Nephrology', 'Neurology', 'Obsterics&Gynecology-GynecologicOnco', 'Obstetrics', 'ObstetricsandGynecology', 'Oncology', 'Ophthalmology', 'Orthopedics', 'Orthopedics-Reconstructive', 'Osteopath', 'Otolaryngology', 'OutreachServices', 'Pathology', 'Pediatrics', 'Pediatrics-CriticalCare', 'Pediatrics-EmergencyMedicine', 'Pediatrics-Endocrinology', 'Pediatrics-Hematology-Oncology', 'Pediatrics-Pulmonology', 'PhysicalMedicineandRehabilitation', 'PhysicianNotFound', 'Podiatry', 'Psychiatry', 'Psychology', 'Pulmonology', 'Radiologist', 'Radiology', 'Surgeon', 'Surgery-Cardiovascular', 'Surgery-Cardiovascular/Thoracic', 'Surgery-Colon&Rectal', 'Surgery-General', 'Surgery-Neuro', 'Surgery-Pediatric', 'Surgery-Plastic', 'Surgery-Plasticw

In [105]:
medical_speciality = sorted([sanitize2(s) for s in set(X_all[:,10])])
medical_speciality_onehot = onehot(medical_speciality)
print(medical_speciality_onehot.shape)

(53, 3, 51)


Number of Lab Procedures

In [21]:
print(set(X_all[:,11]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 101, 103, 104, 106, 107, 108, 109, 113, 114, 120}


Number of Procedures

In [22]:
print(set(X_all[:,12]))

{0, 1, 2, 3, 4, 5, 6}


Number of Medications

In [23]:
print(set(X_all[:,13]))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 65, 67, 74, 75, 81}


Number Outpatient

In [24]:
print(set(X_all[:,14]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 27, 36}


Number Emergency

In [25]:
print(set(X_all[:,15]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 42, 13}


Number Inpatient

In [26]:
print(set(X_all[:,16]))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


Diagnoses

In [27]:
print(set(X_all[:,17])|set(X_all[:,18])|set(X_all[:,19]))

{'847', '619', 'E932', '299', '421', '550', '642', 'V58', '871', '250.2', '397', '745', '318', '272', '475', '991', '591', '751', '598', '429', '288', '512', '296', '295', '462', '464', '836', '967', '78', '345', '420', '303', '622', '355', '473', '966', '530', '82', '381', 'E812', '572', '425', '586', '533', 'V49', '348', '428', '840', '431', '756', '782', '281', '309', '867', '266', '796', '193', '643', '507', '414', '648', '716', '805', '785', '250.11', '824', '250.82', '844', 'E944', '710', '534', '305', '250.8', '416', '714', 'V10', '789', '358', 'V85', '607', '823', '862', '158', '199', '604', '312', '42', '250', '617', '633', '135', '250.7', '791', '557', 'V09', 'V12', '223', '398', '160', '780', '316', '694', 'E934', 'E938', 'V18', 'V27', '480', '715', '661', '280', '413', '294', '741', 'V63', '810', '75', '9', '112', '208', '191', '995', '282', '812', 'E950', '701', '781', '584', '814', '581', '?', '513', '645', '998', '735', '969', '412', '700', '252', '370', '758', '188', '2

Number of Diagnoses

In [28]:
print(set(X_all[:,20]))

{1, 2, 3, 4, 5, 6, 7, 8, 9}


Max Glucose Serum

In [29]:
print(set(X_all[:,21]))

{'Norm', '>300', 'None', '>200'}


In [111]:
max_glu_serum = {'Norm':0, '>300':3, 'None':1, '>200':2}

A1Cresult

In [30]:
print(set(X_all[:,22]))

{'>7', 'Norm', '>8', 'None'}


In [112]:
a1c_res = {'>7': 2, 'Norm': 0, '>8': 3, 'None': 1}

Metformin - Metformin Pioglitazone

In [31]:
print(set(X_all[:,23]))

{'Steady', 'No', 'Down', 'Up'}


In [50]:
drug_change = {'Down': -1, 'No': 0, 'Steady': 0, 'Up': 1}

Change

In [33]:
print(set(X_all[:,46]))

{'No', 'Ch'}


In [113]:
change = {'No': 0, 'Ch': 1}

Diabetes Medication

In [34]:
print(set(X_all[:,47]))

{'No', 'Yes'}


In [114]:
diabetes_med = {'No': 0, 'Yes': 1}

Diagnoses Description

In [35]:
print(set(X_all[:,48]))

{nan, 'Leukoplakia of penis', 'Nonspecific abnormal findings in cerebrospinal fluid', 'Hypertrophy (benign) of prostate without urinary obstruction and other lower urinary tract symptom (LUTS)', 'Malignant neoplasm of nipple and areola of female breast', 'Urinary-genital tract fistula, female', 'Nausea and vomiting', 'Hereditary spherocytosis', 'Intestinal infection due to E. coli, unspecified', 'Alteration of consciousness', 'Pneumonia due to adenovirus', 'Anorexia', 'Pruritus ani', 'Oligohydramnios, unspecified as to episode of care or not applicable', 'Benign neoplasm of ovary', 'Open wound of hand except finger(s) alone, without mention of complication', 'Tympanosclerosis, unspecified as to involvement', 'Malignant neoplasm of rectosigmoid junction', 'Dermatophytosis of scalp and beard', 'Trigeminal nerve disorders', 'Foreign body in pharynx', 'Late effects of cerebrovascular disease, cognitive deficits', 'Acute appendicitis with generalized peritonitis', 'Acute duodenal ulcer with